
---

# 🧪 Лабораторная работа №3

# Выполнил: Шамсутдинов Рустам БВТ2201

## 🎯 Цель

Сравнить **ASR-движки** — *Whisper*, *Vosk* и *Giga AM* —
по метрикам **WER**, **CER**, **RTF**,
а также оценить влияние **bias prompt (контекста)** в *Whisper*.

---

## 📋 Что нужно сделать

1. **Подготовить данные**

   * 10 аудиофайлов формата **WAV** (16 кГц, mono)
   * Эталонные транскрипты в `.txt`

2. **Провести распознавание речи**

   * Прогнать каждое аудио через:

     * 🟢 Whisper
     * 🔵 Vosk
     * 🟣 Giga AM

3. **Посчитать метрики**

   * **WER (Word Error Rate)**
   * **CER (Character Error Rate)**
   * **RTF (Real-Time Factor)**
   * Для каждого файла вычислить и сравнить результаты.

4. **Оценить влияние bias prompt**

   * Записать **5–10 коротких аудио** с доменными терминами.
   * Для *Whisper* выполнить два прогона:

     1. Без `prompt`
     2. С `initial_prompt`, содержащим перечень терминов
   * Сравнить значения **WER/CER** в обоих случаях.

---

## 🔗 Полезные ссылки и материалы

* [SYSTRAN Faster Whisper](https://github.com/SYSTRAN/faster-whisper)
* [Salute Developers GigaAM](https://github.com/salute-developers/GigaAM)
* [Vosk API](https://github.com/alphacep/vosk-api)
* [Hugging Face Audio Course — Chapter 5: Evaluation](https://huggingface.co/learn/audio-course/ru/chapter5/evaluation)

---
